<a href="https://colab.research.google.com/github/KamonohashiPerry/PyTorch/blob/master/PyTorch_2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# PyTorchのインストール

In [0]:
# pip install http://download.pytorch.org/whl/cu80/torch-0.4.0-cp36-cp36m-linux_x86_64.whl

In [0]:
# pip install torchvision

In [0]:
# pip install tqdm

インストールできたかの確認

In [0]:
# import torch

In [0]:
# torch.tensor([1, 2, 3]).to("cuda:0")

In [9]:
!apt-get install -y -qq software-properties-common python-software-properties module-init-tools
!add-apt-repository -y ppa:alessandro-strada/ppa 2>&1 > /dev/null
!apt-get update -qq 2>&1 > /dev/null
!apt-get -y install -qq google-drive-ocamlfuse fuse

from google.colab import auth
auth.authenticate_user()
from oauth2client.client import GoogleCredentials
creds = GoogleCredentials.get_application_default()
import getpass
!google-drive-ocamlfuse -headless -id={creds.client_id} -secret={creds.client_secret} < /dev/null 2>&1 | grep URL

vcode = getpass.getpass()
!echo {vcode} | google-drive-ocamlfuse -headless -id={creds.client_id} -secret={creds.client_secret}

E: Package 'python-software-properties' has no installation candidate
Selecting previously unselected package google-drive-ocamlfuse.
(Reading database ... 145605 files and directories currently installed.)
Preparing to unpack .../google-drive-ocamlfuse_0.7.14-0ubuntu1~ubuntu18.04.1_amd64.deb ...
Unpacking google-drive-ocamlfuse (0.7.14-0ubuntu1~ubuntu18.04.1) ...
Setting up google-drive-ocamlfuse (0.7.14-0ubuntu1~ubuntu18.04.1) ...
Processing triggers for man-db (2.8.3-2ubuntu0.1) ...
Please, open the following URL in a web browser: https://accounts.google.com/o/oauth2/auth?client_id=32555940559.apps.googleusercontent.com&redirect_uri=urn%3Aietf%3Awg%3Aoauth%3A2.0%3Aoob&scope=https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive&response_type=code&access_type=offline&approval_prompt=force
··········
Please, open the following URL in a web browser: https://accounts.google.com/o/oauth2/auth?client_id=32555940559.apps.googleusercontent.com&redirect_uri=urn%3Aietf%3Awg%3Aoauth%3A2.0%3Aoob&scope

In [10]:
# drive mean root directory of  google drive
!mkdir -p drive
!google-drive-ocamlfuse drive
!ls drive/"Colab Notebooks"/PyTorch

fuse: mountpoint is not empty
fuse: if you are sure this is safe, use the 'nonempty' mount option
FashionMNIST


# Chapter4

In [0]:
import torch
from torch import nn, optim
from torch.utils.data import (Dataset, DataLoader, TensorDataset)
import tqdm

## データの作成

In [2]:
from torchvision.datasets import FashionMNIST
from torchvision import transforms

# 訓練用のデータを取得
# そのままだとPython Imaging Libraryの画像形式でDatasetを作ってしまうので、transforms.ToTensorでTensorに変換する
fashion_mnist_train = FashionMNIST("drive/Colab Notebooks/PyTorch/FashionMNIST",
                                   train=True,
                                   download=True,
                                   transform=transforms.ToTensor())

# 検証用データの取得
fashion_mnist_test = FashionMNIST("drive/Colab Notebooks/PyTorch/FashionMNIST",
                                  train=False,
                                  download=True,
                                  transform=transforms.ToTensor())

# バッチサイズが128のDataLoaderをそれぞれ作成
batch_size = 128
train_loader = DataLoader(fashion_mnist_train,
                          batch_size=batch_size,
                          shuffle=True)

test_loader = DataLoader(fashion_mnist_test,
                          batch_size=batch_size,
                          shuffle=True)

0it [00:00, ?it/s]

26427392it [00:04, 5795878.96it/s]                              


Extracting drive/Colab Notebooks/PyTorch/FashionMNIST/FashionMNIST/raw/train-images-idx3-ubyte.gz to drive/Colab Notebooks/PyTorch/FashionMNIST/FashionMNIST/raw


0it [00:00, ?it/s]

32768it [00:00, 36419.07it/s]                           
0it [00:00, ?it/s]

Extracting drive/Colab Notebooks/PyTorch/FashionMNIST/FashionMNIST/raw/train-labels-idx1-ubyte.gz to drive/Colab Notebooks/PyTorch/FashionMNIST/FashionMNIST/raw


4423680it [00:02, 1675200.29it/s]                            
0it [00:00, ?it/s]

Extracting drive/Colab Notebooks/PyTorch/FashionMNIST/FashionMNIST/raw/t10k-images-idx3-ubyte.gz to drive/Colab Notebooks/PyTorch/FashionMNIST/FashionMNIST/raw


8192it [00:00, 15081.43it/s]            

Extracting drive/Colab Notebooks/PyTorch/FashionMNIST/FashionMNIST/raw/t10k-labels-idx1-ubyte.gz to drive/Colab Notebooks/PyTorch/FashionMNIST/FashionMNIST/raw
Processing...
Done!


## CNNの構築と学習

In [3]:
# (N, C, H, W)形式のTensorを(N, C*H*Wに引き伸ばす層)
# C:color, H:height, W:width
# 畳み込み層の出力をMLPに渡す際に必要
class FlattenLayer(nn.Module):
  def forward(self, x):
    sizes = x.size()
    return x.view(sizes[0], -1)

# 5×5のカーネルを使用し、最初に32個、次に64個のチャンネルを作成する
# BatchNorm2dは画像形式用のBatch Normalization
# Dropout2dは画像形式用のDropout
# 最後にFlattenLayerを挟む
conv_net = nn.Sequential(
    nn.Conv2d(1, 32, 5),
    nn.MaxPool2d(2),
    nn.ReLU(),
    nn.BatchNorm2d(32),
    nn.Dropout2d(0.25),
    nn.Conv2d(32, 64, 5),
    nn.MaxPool2d(2),
    nn.ReLU(),
    nn.BatchNorm2d(64),
    nn.Dropout2d(0.25),
    FlattenLayer()
)

# 畳み込みによって最終的にどうようなサイズになっているかを実際にダミーデータを入れてみて確認する
test_input = torch.ones(1, 1, 28, 28)
conv_output_size = conv_net(test_input).size()[-1]
conv_output_size

1024

In [4]:
# 2層のMLP
mlp = nn.Sequential(
    nn.Linear(conv_output_size, 200),
    nn.ReLU(),
    nn.BatchNorm1d(200),
    nn.Dropout(0.25),
    nn.Linear(200, 10)
)

# 最終的なCNN
net = nn.Sequential(
    conv_net,
    mlp
)

net

Sequential(
  (0): Sequential(
    (0): Conv2d(1, 32, kernel_size=(5, 5), stride=(1, 1))
    (1): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
    (2): ReLU()
    (3): BatchNorm2d(32, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (4): Dropout2d(p=0.25, inplace=False)
    (5): Conv2d(32, 64, kernel_size=(5, 5), stride=(1, 1))
    (6): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
    (7): ReLU()
    (8): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (9): Dropout2d(p=0.25, inplace=False)
    (10): FlattenLayer()
  )
  (1): Sequential(
    (0): Linear(in_features=1024, out_features=200, bias=True)
    (1): ReLU()
    (2): BatchNorm1d(200, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (3): Dropout(p=0.25, inplace=False)
    (4): Linear(in_features=200, out_features=10, bias=True)
  )
)

In [0]:
# 評価のヘルパー関数
def eval_net(net, data_loader, device="cpu"):
  # DropoutやBatchNormを無効化
  net.eval()

  ys = []
  ypreds = []

  for x, y in data_loader:
    # toメソッドで計算を実行するデバイスに転送する
    x = x.to(device)
    y = y.to(device)

    # 確率が最大のクラスを予測
    # ここではforwardの計算だけなので自動微分に必要な処理はoffにして余計な計算を省く
    with torch.no_grad():
      _, y_pred = net(x).max(1)

    ys.append(y)
    ypreds.append(y_pred)

    # ミニバッチごとの予測結果などを1つにまとめる
    ys = torch.cat(ys)
    ypreds = torch.cat(ypreds)

    # 予測精度を計算
    acc = (ys == ypreds).float().sum()/len(ys)
    return acc.item()

# 訓練のヘルパー関数
def train_net(net, train_loader, test_loader,
              optimizer_cls=optim.Adam,
              loss_fn=nn.CrossEntropyLoss(),
              n_iter=10,device="cpu"):
  train_losses = []
  train_acc = []
  val_acc = []
  optimizer = optimizer_cls(net.parameters())

  for epoch in range(n_iter):
    running_loss = 0.0
    # ネットワークを訓練モードにする
    net.train()
    n = 0
    n_acc = 0
    # 非常に時間がかかるのでtqdmを使用してプログレスバーを出す
    for i, (xx, yy) in tqdm.tqdm(enumerate(train_loader),
                                  total = len(train_loader)):
      xx = xx.to(device)
      yy = yy.to(device)

      h = net(xx)
      loss = loss_fn(h, yy)
      optimizer.zero_grad()
      loss.backward()
      optimizer.step()
      running_loss += loss.item()
      n += len(xx)
      _, y_pred = h.max(1)
      n_acc += (yy == y_pred).float().sum().item()
    train_losses.append(running_loss / i)

    # 訓練データの予測精度
    train_acc.append(n_acc / n)

    # 検証データの予測精度
    val_acc.append(eval_net(net, test_loader, device))

    # このepochでの結果の表示
    print(epoch, train_losses[-1], train_acc[-1], val_acc[-1], flush=True)


In [13]:
# ネットワークの全パラメータをGPUに転送
net.to("cuda:0")

# 訓練を実行
train_net(net, train_loader, test_loader, n_iter=20,device="cuda:0")

100%|██████████| 469/469 [00:06<00:00, 71.69it/s]

0 0.45682357754717523 0.8358666666666666 0.8359375



100%|██████████| 469/469 [00:06<00:00, 73.04it/s]

1 0.3240702795740376 0.8827666666666667 0.859375



100%|██████████| 469/469 [00:06<00:00, 71.66it/s]

...

17 0.16410207822441292 0.9399166666666666 0.90625


100%|██████████| 469/469 [00:06<00:00, 73.49it/s]

18 0.15877481164713192 0.9400666666666667 0.875



100%|██████████| 469/469 [00:06<00:00, 73.40it/s]

19 0.1558684255195479 0.9421333333333334 0.9375


## 転移学習

In [12]:
!wget https://github.com/lucidfrontier45/PyTorch-Book/raw/master/data/taco_and_burrito.tar.gz

--2019-12-01 06:37:08--  https://github.com/lucidfrontier45/PyTorch-Book/raw/master/data/taco_and_burrito.tar.gz
Resolving github.com (github.com)... 140.82.113.3
Connecting to github.com (github.com)|140.82.113.3|:443... connected.
HTTP request sent, awaiting response... 302 Found
Location: https://raw.githubusercontent.com/lucidfrontier45/PyTorch-Book/master/data/taco_and_burrito.tar.gz [following]
--2019-12-01 06:37:09--  https://raw.githubusercontent.com/lucidfrontier45/PyTorch-Book/master/data/taco_and_burrito.tar.gz
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 151.101.0.133, 151.101.64.133, 151.101.128.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|151.101.0.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 15292798 (15M) [application/octet-stream]
Saving to: ‘taco_and_burrito.tar.gz’

taco_and_burrito.ta 100%[===================>]  14.58M  96.1MB/s    in 0.2s    

2019-12-01 06:37:11 (96.1 MB/s) - ‘t

In [0]:
# !tar -zxvf taco_and_burrito.tar.gz

In [0]:
# !mv taco_and_burrito drive/"Colab Notebooks"/PyTorch/taco_and_burrito

In [0]:
from torchvision.datasets import ImageFolder
from torchvision import transforms

# ImageFolder関数を使用してDatasetを作成する
train_imgs = ImageFolder(
    "drive/Colab Notebooks/PyTorch/taco_and_burrito/train",
    transform = transforms.Compose([transforms.RandomCrop(224),
                                        transforms.ToTensor()])
)

test_imgs = ImageFolder(
    "drive/Colab Notebooks/PyTorch/taco_and_burrito/test",
    transform = transforms.Compose([transforms.CenterCrop(224),
                                       transforms.ToTensor()])
)

# DataLoaderを作成
train_loader = DataLoader(train_imgs, batch_size=32, shuffle=True)
test_loader = DataLoader(test_imgs, batch_size=32, shuffle=False) # テストの方はシャッフルしない。

In [17]:
print(train_imgs.classes)

['burrito', 'taco']


In [18]:
print(train_imgs.class_to_idx)

{'burrito': 0, 'taco': 1}


In [19]:
from torchvision import models

# 事前学習済みのresnet18をロード
net = models.resnet18(pretrained=True)

# 全てのパラメータを自動微分対象外にする
for p in net.parameters():
  p.requires_grad=False

# 最後の線形層を付け替える
fc_input_dim = net.fc.in_features
net.fc = nn.Linear(fc_input_dim, 2)

Downloading: "https://download.pytorch.org/models/resnet18-5c106cde.pth" to /root/.cache/torch/checkpoints/resnet18-5c106cde.pth
100%|██████████| 44.7M/44.7M [00:00<00:00, 218MB/s]


In [0]:
# モデルの訓練関数

def eval_net(net, data_loader, device="cpu"):
  # DropoutやBatcNormを無効化
  net.eval()

  ys = []
  ypreds = []

  for x, y in data_loader:
    # toメソッドで計算を実行するデバイスに転送する
    x = x.to(device)
    y = y.to(device)

    # 確率が最大のクラスを予測
    # ここではforwardの計算だけなので自動微分に必要な処理はoffにして余計な計算を省く
    with torch.no_grad():
      _, y_pred = net(x).max(1)

    ys.append(y)
    ypreds.append(y_pred)

  # ミニバッチごとの予測結果などを1つにまとめる
  ys = torch.cat(ys)
  ypreds = torch.cat(ypreds)

  # 予測精度を計算
  acc = (ys == ypreds).float().sum() / len(ys)
  return acc.item()


def train_net(net, train_loader, test_loader,
              only_fc = True,
              optimizer_cls=optim.Adam,
              loss_fn=nn.CrossEntropyLoss(),
              n_iter=10,
              device="cpu"):
  train_losses = []
  train_acc = []
  val_acc = []
  if only_fc:
    # 最後の線形層のパラメータのみをoptimizerに渡す
    optimizer = optimizer_cls(net.fc.parameters())
  else:
    optimizer = optimizer_cls(net.parameters())

  for epoch in range(n_iter):
    running_loss = 0.0
    # ネットワークを訓練モードにする
    net.train()
    n = 0
    n_acc = 0

    # 非常に時間がかかるのでtqdmを使用してプログレスバーを出す
    for i, (xx, yy) in tqdm.tqdm(enumerate(train_loader),
                                 total=len(train_loader)):
      xx = xx.to(device)
      yy = yy.to(device)

      h = net(xx)
      loss = loss_fn(h, yy)

      optimizer.zero_grad()
      loss.backward()

      optimizer.step()
      running_loss += loss.item()

      n += len(xx)
      _, y_pred = h.max(1)
      n_acc += (yy == y_pred).float().sum().item()

    train_losses.append(running_loss / i)

    # 訓練データの予測精度
    train_acc.append(n_acc / n)

    # 検証データの予測精度
    val_acc.append(eval_net(net, test_loader, device))

    # このepochでの結果を表示
    print(epoch, train_losses[-1], train_acc[-1],
          val_acc[-1], flush=True)

In [21]:
# ネットワークの全パラメータをGPUに転送
net.to("cuda:0")

# 訓練を実行
train_net(net, train_loader, test_loader, n_iter=20, device="cuda:0")

100%|██████████| 23/23 [00:04<00:00,  5.67it/s]


0 0.7108825228431008 0.5702247191011236 0.6833333969116211


100%|██████████| 23/23 [00:03<00:00,  5.91it/s]


1 0.5333546210419048 0.7654494382022472 0.8500000238418579


100%|██████████| 23/23 [00:03<00:00,  6.01it/s]


...

17 0.28614081781018863 0.898876404494382 0.8666667342185974


100%|██████████| 23/23 [00:03<00:00,  6.75it/s]


18 0.2856492914936759 0.8918539325842697 0.8833333849906921


100%|██████████| 23/23 [00:03<00:00,  6.62it/s]


19 0.2961136874827472 0.875 0.8666667342185974


In [0]:
# fine tuningの別の書き方

class IdentityLayer(nn.Module):
  def forward(self, x):
    return x
  
net = models.resnet18(pretrained=True)
for p in net.parameters():
  p.requires_grad=False
net.fc = IdentityLayer()

In [24]:
# 転移学習を使わない場合どうなるか

conv_net = nn.Sequential(
    nn.Conv2d(3, 32, 5),
    nn.MaxPool2d(2),
    nn.ReLU(),
    nn.BatchNorm2d(32),
    nn.Conv2d(32, 64, 5),
    nn.MaxPool2d(2),
    nn.ReLU(),
    nn.BatchNorm2d(64),
    nn.Conv2d(64, 128, 5),
    nn.MaxPool2d(2),
    nn.ReLU(),
    nn.BatchNorm2d(128),
    FlattenLayer()
)

# 畳み込みによってどのようなサイズになっているかを実際にデータを入れて確認する
test_input = torch.ones(1, 3, 224, 224)
conv_output_size = conv_net(test_input).size()[-1]
conv_output_size

73728

In [0]:
# 最終的なCNN
net = nn.Sequential(
    conv_net,
    nn.Linear(conv_output_size, 2)
)

# 訓練を実行
train_net(net, train_loader, test_loader, n_iter=10,only_fc=False)

100%|██████████| 23/23 [01:55<00:00,  4.00s/it]


0 4.477879944172773 0.5828651685393258 0.6166666746139526


100%|██████████| 23/23 [01:54<00:00,  3.99s/it]


1 5.907462442463094 0.5730337078651685 0.699999988079071


100%|██████████| 23/23 [01:54<00:00,  4.00s/it]


...

100%|██████████| 23/23 [01:54<00:00,  4.02s/it]


3 5.080085786906156 0.6053370786516854 0.6833333373069763


100%|██████████| 23/23 [01:55<00:00,  3.99s/it]


4 5.568163015625694 0.6320224719101124 0.5833333134651184


 48%|████▊     | 11/23 [00:56<01:01,  5.16s/it]